In [68]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import skfolio
from skfolio import Population, RiskMeasure
from skfolio.datasets import load_sp500_dataset
from skfolio.optimization import InverseVolatility, MeanRisk, ObjectiveFunction
from skfolio.preprocessing import prices_to_returns
from data_loader import index_history
from data_loader import ticker_prices
from plotly.io import show
import requests

import warnings
warnings.filterwarnings('ignore')

In [66]:
# Определяем параметры
tickers = ['AFKS', 'AFLT', 'ALRS', 'ASTR', 'BSPB', 'CBOM', 'CHMF', 'ENPG', 'FEES', 'FLOT', 'GAZP', 'GMKN', 'HEAD', 'HYDR', 'IRAO', 'LKOH', 'MAGN', 'MDMG', 'MOEX', 'MSNG', 'MTLR', 'MTSS', 'NLMK', 'NVTK', 'PHOR', 'PIKK', 'PLZL', 'POSI', 'RENI', 'ROSN', 'RTKM', 'RUAL', 'SBER', 'SBERP', 'SELG', 'SNGS', 'SNGSP', 'SVCB', 'T', 'TATN', 'TATNP', 'TRNFP', 'UGLD', 'UPRO', 'VKCO', 'VTBR', 'YDEX']
#tickers = ['MCFTR', 'RGBITR', 'MESMTR']
risk_free_rate = 0.05
end_date = datetime.today()
start_date = end_date - timedelta(days=15 * 365)

In [69]:
# Загружаем исторические данные
#data = index_history(tickers, start_date=start_date, end_date=end_date)
data = ticker_prices(tickers)
data = data.set_index('TRADEDATE')

In [135]:
data_filtered = data.drop(['YDEX', 'HEAD', 'SVCB', 'UGLD', 'ASTR'], axis=1)

In [137]:
# Convert prices to returns
returns = prices_to_returns(data_filtered)

model = MeanRisk(
    risk_measure=RiskMeasure.STANDARD_DEVIATION,
    objective_function=ObjectiveFunction.MAXIMIZE_RATIO,
    portfolio_params=dict(name="Max Sharpe"),
    min_weights=0.015,
)

In [136]:
data_filtered.describe().loc['count']

AFKS     4332.0
AFLT     5290.0
ALRS     3356.0
BSPB     4268.0
CBOM     2457.0
CHMF     4955.0
ENPG     1284.0
FEES     4179.0
FLOT     1125.0
GAZP     4811.0
GMKN     4572.0
HYDR     4227.0
IRAO     3833.0
LKOH     5411.0
MAGN     4813.0
MDMG     1081.0
MOEX     3049.0
MSNG     5385.0
MTLR     4079.0
MTSS     5283.0
NLMK     4751.0
NVTK     4667.0
PHOR     3426.0
PIKK     4441.0
PLZL     4733.0
POSI      820.0
RENI      856.0
ROSN     4689.0
RTKM     5371.0
RUAL     2516.0
SBER     4439.0
SBERP    4439.0
SELG     3307.0
SNGS     5425.0
SNGSP    5425.0
T        1340.0
TATN     5831.0
TATNP    5831.0
TRNFP    4317.0
UPRO     2203.0
VKCO      799.0
VTBR     4395.0
Name: count, dtype: float64

In [138]:
data_filtered.describe()

,AFKS,AFLT,ALRS,BSPB,CBOM,CHMF,ENPG,FEES,FLOT,GAZP,...,SELG,SNGS,SNGSP,T,TATN,TATNP,TRNFP,UPRO,VKCO,VTBR
count,4332.000000,5290.000000,3356.000000,4268.000000,2457.000000,4955.000000,1284.000000,4179.000000,1125.000000,4811.00000,...,3307.000000,5425.000000,5425.000000,1340.000000,5831.000000,5831.000000,4317.000000,2203.000000,799.000000,4395.000000
mean,21.039697,67.398302,69.838649,94.435377,5.754168,715.362101,572.978660,0.174593,86.027493,185.62313,...,24.375442,28.584391,28.084481,3151.627015,296.743731,228.494375,1085.652754,2.395076,490.716020,259.353984
std,8.996065,36.556557,26.670139,86.021868,1.386719,425.937480,180.464802,0.092298,30.240708,60.54218,...,22.534366,6.313900,13.632634,1571.749187,229.916619,219.140422,567.692682,0.491445,143.748712,118.884009
min,3.600000,20.200000,21.999000,23.130000,3.620000,58.650000,278.400000,0.032870,29.920000,86.60000,...,0.000000,8.701000,4.541000,831.800000,14.350000,8.910000,60.500000,1.021000,237.800000,64.210000
25%,13.505000,42.015000,52.187500,44.940000,4.575000,357.100000,429.837500,0.097715,60.380000,141.96000,...,7.495000,25.085000,16.197000,2255.000000,124.300000,69.755000,533.510000,2.041000,357.700000,178.987500
50%,19.828000,57.550000,70.750000,57.955000,5.780000,624.990000,520.000000,0.166940,89.850000,163.61000,...,10.100000,28.195000,27.480000,2746.500000,207.760000,110.270000,1238.000000,2.610000,478.600000,241.250000
75%,26.900000,78.100000,87.497500,99.685000,6.765000,978.300000,731.500000,0.214570,102.190000,220.81000,...,45.000000,32.295000,37.785000,3470.500000,485.050000,417.900000,1590.000000,2.765000,605.900000,342.075000
max,48.000000,225.000000,150.300000,409.760000,10.623000,2006.000000,998.000000,0.454000,149.300000,389.82000,...,91.200000,54.170000,72.425000,8345.000000,837.000000,777.500000,2310.000000,3.140000,982.800000,1207.500000


In [139]:
# Run optimization
model.fit(returns)

MeanRisk(min_weights=0.015, objective_function=MAXIMIZE_RATIO,
         portfolio_params={'name': 'Max Sharpe'},
         risk_measure=Standard Deviation)

In [140]:
print("Optimal Weights:")
for ticker, weight in zip(tickers, model.weights_):
    print(f"{ticker}: {weight:.4f}")

Optimal Weights:
AFKS: 0.0150
AFLT: 0.0150
ALRS: 0.0150
ASTR: 0.3850
BSPB: 0.0150
CBOM: 0.0150
CHMF: 0.0150
ENPG: 0.0150
FEES: 0.0150
FLOT: 0.0150
GAZP: 0.0150
GMKN: 0.0150
HEAD: 0.0150
HYDR: 0.0150
IRAO: 0.0150
LKOH: 0.0150
MAGN: 0.0150
MDMG: 0.0150
MOEX: 0.0150
MSNG: 0.0150
MTLR: 0.0150
MTSS: 0.0150
NLMK: 0.0150
NVTK: 0.0150
PHOR: 0.0150
PIKK: 0.0150
PLZL: 0.0150
POSI: 0.0150
RENI: 0.0150
ROSN: 0.0150
RTKM: 0.0150
RUAL: 0.0150
SBER: 0.0150
SBERP: 0.0150
SELG: 0.0150
SNGS: 0.0150
SNGSP: 0.0150
SVCB: 0.0150
T: 0.0150
TATN: 0.0150
TATNP: 0.0150
TRNFP: 0.0150


In [141]:
benchmark = InverseVolatility(portfolio_params=dict(name="Inverse Vol"))
benchmark.fit(returns)

InverseVolatility(portfolio_params={'name': 'Inverse Vol'})

In [142]:
model_pred = model.predict(returns)
benchmark_pred = benchmark.predict(returns)

In [143]:
population = Population([model_pred, benchmark_pred])

In [144]:
fig = population.plot_cumulative_returns()
# show(fig) is only used for the documentation sticker.
show(fig)

In [145]:
population.summary()

,Max Sharpe,Inverse Vol
Mean,0.099%,0.014%
Annualized Mean,24.84%,3.58%
Variance,0.043%,0.032%
Annualized Variance,10.87%,8.12%
Semi-Variance,0.025%,0.021%
Annualized Semi-Variance,6.29%,5.21%
Standard Deviation,2.08%,1.80%
Annualized Standard Deviation,32.97%,28.50%
Semi-Deviation,1.58%,1.44%
Annualized Semi-Deviation,25.08%,22.83%


In [59]:
population.composition()

,Max Sharpe,Inverse Vol
asset,,
RGBITR,0.793837,0.562867
MCFTR,0.206163,0.206490
MESMTR,0.000000,0.230643


In [62]:
returns.corr()

,MCFTR,RGBITR,MESMTR
MCFTR,1.000000,0.464874,0.861317
RGBITR,0.464874,1.000000,0.499017
MESMTR,0.861317,0.499017,1.000000
